<a href="https://colab.research.google.com/github/Hyemmingway/ai_teamplay/blob/main/%EC%9D%B8%EA%B3%B5%EC%A7%80%EB%8A%A5_KSL_1203_mpyes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 0. 모듈 불러오기

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import os
import glob
from pathlib import Path

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import random
import math

import pickle
import cv2
#import mediapipe as mp

from tensorflow.keras.preprocessing.image import ImageDataGenerator

## 1. 데이터 확인

In [4]:
#한 폴더당 중심 동작 20개씩 뽑음
select_df=pd.read_csv('/content/drive/MyDrive/2022-2 ipynb/AISTAT/num5 data/DF_select_mp_20 (1).csv')
select_df

,name,fold,class,num,path,label
0,00_01.0045.jpg,00_01,1,45,/content/drive/MyDrive/2022-2 ipynb/AISTAT/num...,hi
1,00_01.0046.jpg,00_01,1,46,/content/drive/MyDrive/2022-2 ipynb/AISTAT/num...,hi
2,00_01.0047.jpg,00_01,1,47,/content/drive/MyDrive/2022-2 ipynb/AISTAT/num...,hi
3,00_01.0048.jpg,00_01,1,48,/content/drive/MyDrive/2022-2 ipynb/AISTAT/num...,hi
4,00_01.0049.jpg,00_01,1,49,/content/drive/MyDrive/2022-2 ipynb/AISTAT/num...,hi
...,...,...,...,...,...,...
3595,19_77.0072.jpg,19_77,77,72,/content/drive/MyDrive/2022-2 ipynb/AISTAT/num...,care
3596,19_77.0073.jpg,19_77,77,73,/content/drive/MyDrive/2022-2 ipynb/AISTAT/num...,care
3597,19_77.0074.jpg,19_77,77,74,/content/drive/MyDrive/2022-2 ipynb/AISTAT/num...,care
3598,19_77.0075.jpg,19_77,77,75,/content/drive/MyDrive/2022-2 ipynb/AISTAT/num...,care


In [5]:
datagen = ImageDataGenerator(rescale=1. / 255, validation_split=0.25)
train_generator = datagen.flow_from_dataframe(
          dataframe=select_df,
          batch_size=128,
          class_mode='categorical',
          shuffle=True,
          x_col='path',
          y_col='label',
          subset = 'training')

valid_generator = datagen.flow_from_dataframe(
          dataframe=select_df,
          batch_size=128,
          class_mode='categorical',
          shuffle=True,
          x_col='path',
          y_col='label',
          subset = 'validation')

/usr/local/lib/python3.8/dist-packages/keras/preprocessing/image.py:989: UserWarning: Found 2409 invalid image filename(s) in x_col="path". These filename(s) will be ignored.
  warnings.warn('Found {} invalid image filename(s) in x_col="{}". '


Found 894 validated image filenames belonging to 10 classes.
Found 297 validated image filenames belonging to 10 classes.


### CNN

In [6]:
from tensorflow.keras import models, layers, Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Activation
from tensorflow.keras.layers import Dense, Dropout, Flatten, BatchNormalization
from tensorflow import keras

In [7]:
from keras.callbacks import ModelCheckpoint

In [8]:
pool_size = 2
    
model1 = Sequential([
  keras.Input(shape=(256,256,3)),
  Conv2D(filters = 30, kernel_size = (3,3), padding='same'),  
  BatchNormalization(),                                       
  Activation('relu'),
  Conv2D(filters = 30, kernel_size = (3,3), padding='same'),  
  BatchNormalization(),                                       
  Activation('relu'),                                           
  MaxPooling2D(pool_size=pool_size),                         

  Conv2D(filters = 50, kernel_size = (3,3), padding='same'),
  BatchNormalization(),
  Activation('relu'),
  Conv2D(filters = 50, kernel_size = (3,3), padding='same'),  
  BatchNormalization(),                                       
  Activation('relu'),   
  MaxPooling2D(pool_size=pool_size),                         

  Conv2D(filters = 80, kernel_size = (3,3), padding='same'),
  BatchNormalization(),
  Activation('relu'),
  Conv2D(filters = 80, kernel_size = (3,3), padding='same'),  
  BatchNormalization(),                                       
  Activation('relu'),   
  MaxPooling2D(pool_size=pool_size),                         

  Flatten(),
  Dense(50),
  Activation('relu'),                                          
  
  Dense(10, activation='softmax'),
])

model1.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 256, 256, 30)      840       
                                                                 
 batch_normalization (BatchN  (None, 256, 256, 30)     120       
 ormalization)                                                   
                                                                 
 activation (Activation)     (None, 256, 256, 30)      0         
                                                                 
 conv2d_1 (Conv2D)           (None, 256, 256, 30)      8130      
                                                                 
 batch_normalization_1 (Batc  (None, 256, 256, 30)     120       
 hNormalization)                                                 
                                                                 
 activation_1 (Activation)   (None, 256, 256, 30)      0

In [9]:
model1.compile(
  optimizer='adam',
  loss='categorical_crossentropy',
  metrics=['accuracy'],
)

In [10]:
#cnn
history1= model1.fit(
    train_generator,
    epochs=30,
    validation_data=valid_generator,
    shuffle=True)

Epoch 1/30
7/7 [==============================] - 28s 3s/step - loss: 16.0383 - accuracy: 0.2271 - val_loss: 2.5114 - val_accuracy: 0.0741
Epoch 2/30
7/7 [==============================] - 7s 980ms/step - loss: 3.9985 - accuracy: 0.2707 - val_loss: 3.7040 - val_accuracy: 0.0741
Epoch 3/30
7/7 [==============================] - 7s 962ms/step - loss: 2.1141 - accuracy: 0.4195 - val_loss: 2.3779 - val_accuracy: 0.1818
Epoch 4/30
7/7 [==============================] - 7s 946ms/step - loss: 1.6148 - accuracy: 0.5067 - val_loss: 2.3212 - val_accuracy: 0.1886
Epoch 5/30
7/7 [==============================] - 7s 943ms/step - loss: 1.3174 - accuracy: 0.5839 - val_loss: 2.3540 - val_accuracy: 0.1448
Epoch 6/30
7/7 [==============================] - 7s 937ms/step - loss: 1.1269 - accuracy: 0.6141 - val_loss: 2.3239 - val_accuracy: 0.0370
Epoch 7/30
7/7 [==============================] - 7s 941ms/step - loss: 1.0166 - accuracy: 0.6767 - val_loss: 2.3300 - val_accuracy: 0.1448
Epoch 8/30
7/7 [=====

### CNN+LSTM

In [11]:
from keras.layers import Dense, Dropout, Flatten, Conv2D, BatchNormalization, MaxPooling2D,\
                        Permute, TimeDistributed, Bidirectional,GRU, LeakyReLU, SimpleRNN,\
LSTM, GlobalAveragePooling2D, SeparableConv2D, ZeroPadding2D, Reshape
from keras import models
from keras import layers
import tensorflow as tf

In [12]:
Model = Sequential()

Model.add(SeparableConv2D(12,(3,3),activation="relu",
                 input_shape=(256,256,3)))
Model.add(BatchNormalization())
Model.add(MaxPooling2D((2,2)))

#
Model.add(SeparableConv2D(24,(3,3),
                 activation="relu",padding="same"))
Model.add(Dropout(0.2))
Model.add(MaxPooling2D((2,2)))

#
Model.add(TimeDistributed(Flatten()))
Model.add(Bidirectional(LSTM(64,
                                  return_sequences=True,
                                  dropout=0.5,
                                  recurrent_dropout=0.5)))
Model.add(Bidirectional(LSTM(64,
                                  return_sequences=True,
                                  dropout=0.5,
                                  recurrent_dropout=0.5)))

#
Model.add(Flatten())
Model.add(Dense(128,activation="relu"))
Model.add(Dropout(0.5))
Model.add(Dense(10,activation="sigmoid"))

In [13]:
Model.compile(
  optimizer='adam',
  loss='categorical_crossentropy',
  metrics=['accuracy'],
)

In [16]:
checkpoint_filepath = '/content/drive/MyDrive/2022-2 ipynb/AISTAT/num5 data' + 'best_model_{val_accuracy:.4f}.h5'
model_checkpoint_callback = ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True,
    verbose=1)

In [17]:
history2= Model.fit(
    train_generator,
    epochs=30,
    validation_data=valid_generator,
    shuffle=True)

Epoch 1/30
7/7 [==============================] - 14s 2s/step - loss: 1.9191 - accuracy: 0.3221 - val_loss: 2.2672 - val_accuracy: 0.1852
Epoch 2/30
7/7 [==============================] - 14s 2s/step - loss: 1.5069 - accuracy: 0.4821 - val_loss: 2.2871 - val_accuracy: 0.0606
Epoch 3/30
7/7 [==============================] - 15s 2s/step - loss: 1.0642 - accuracy: 0.6499 - val_loss: 2.3080 - val_accuracy: 0.2357
Epoch 4/30
7/7 [==============================] - 14s 2s/step - loss: 0.7891 - accuracy: 0.7349 - val_loss: 2.3194 - val_accuracy: 0.2020
Epoch 5/30
7/7 [==============================] - 14s 2s/step - loss: 0.5551 - accuracy: 0.8244 - val_loss: 2.3754 - val_accuracy: 0.0909
Epoch 6/30
7/7 [==============================] - 13s 2s/step - loss: 0.3624 - accuracy: 0.8781 - val_loss: 2.3267 - val_accuracy: 0.0875
Epoch 7/30
7/7 [==============================] - 13s 2s/step - loss: 0.3166 - accuracy: 0.8792 - val_loss: 2.2905 - val_accuracy: 0.1481
Epoch 8/30
7/7 [==================

### VGG16
- 하려다가 실패함

In [ ]:
from keras.applications.vgg16 import VGG16

In [ ]:
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(256,256,3), classes=10)

In [ ]:
base_model.trainable = False

In [ ]:
base_model.compile(
  optimizer='adam',
  loss='categorical_crossentropy',
  metrics=['accuracy'],
)

In [ ]:
history3= base_model.fit(
    train_generator,
    epochs=10,
    validation_data=valid_generator,
    shuffle=True)